In [1]:
import gym
from gym.utils import play
from IL_Problem.base.utils.callbacks import Callbacks, load_expert_memories
from RL_Agent import dddqn_agent, ppo_agent_discrete_parallel
from RL_Agent.base.utils.networks import networks
from IL_Problem.base.utils.networks import networks_dictionaries as il_networks
from RL_Problem import rl_problem as rl_p
from IL_Problem.deepirl import DeepIRL
from IL_Problem.gail import GAIL
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input, MaxPooling2D
import numpy as np

/home/serch/anaconda3/envs/capoir/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


pygame 2.1.0 (SDL 2.0.16, Python 3.7.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


2021-12-10 10:22:44.260468: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2021-12-10 10:22:44.286218: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2799925000 Hz
2021-12-10 10:22:44.286658: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557eaa8d4ed0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-12-10 10:22:44.286680: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-12-10 10:22:44.286788: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [2]:
env_name ="SpaceInvaders-v0"
env = gym.make(env_name)

A.L.E: Arcade Learning Environment (version +978d2ce)
[Powered by Stella]


In [3]:
cb = Callbacks()

In [4]:
play.play(env, zoom=3, callback=cb.remember_callback)

In [5]:
exp_path = "expert_demonstrations/SpaceInvaders_expert.pkl"
cb.save_memories(exp_path)

data ready
data saved


In [6]:
# Función para preprocesar las imágenes
def atari_preprocess(obs):
    # Crop and resize the image
    obs = obs[20:200:2, ::2]

    # Convert the image to greyscale
    obs = obs.mean(axis=2)

    # normalize between from 0 to 1
    obs = obs / 255.
    obs = obs[:, :, np.newaxis]
    return obs

In [7]:
# Guardamos las dimensiones del estado una vez preprocesado, es necesario que el tercer eje marque el número de canales
state_size = (90, 80, 1)

In [8]:
net_architecture = networks.ppo_net(actor_conv_layers=2,
                                    actor_kernel_num=[8, 8],
                                    actor_kernel_size=[3, 3],
                                    actor_kernel_strides=[2, 2],
                                    actor_conv_activation=['relu', 'relu'],
                                    actor_dense_layers=2,
                                    actor_n_neurons=[128, 128],
                                    actor_dense_activation=['relu', 'relu'],

                                    critic_conv_layers=2,
                                    critic_kernel_num=[8, 8],
                                    critic_kernel_size=[3, 3],
                                    critic_kernel_strides=[2, 2],
                                    critic_conv_activation=['relu', 'relu'],
                                    critic_dense_layers=2,
                                    critic_n_neurons=[128, 128],
                                    critic_dense_activation=['relu', 'relu'],
                                    use_custom_network=False)

In [9]:
agent = ppo_agent_discrete_parallel.Agent(actor_lr=1e-4,
                                              critic_lr=1e-4,
                                              batch_size=128,
                                              epsilon=0.9,
                                              epsilon_decay=0.97,
                                              epsilon_min=0.15,
                                              memory_size=128,
                                              net_architecture=net_architecture,
                                              n_stack=5,
                                              img_input=True,
                                              state_size=(90, 80, 1)
                                              )

In [10]:
rl_problem = rl_p.Problem(env, agent)
rl_problem.preprocess = atari_preprocess

Process Process-8:
Process Process-7:
Process Process-3:
Process Process-4:
Process Process-1:
Process Process-6:
Process Process-5:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/serch/anaconda3/envs/capoir/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/serch/anaconda3/envs/capoir/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/serch/anaconda3/envs/capoir/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/serch/anaconda3/envs/capoir/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/serch/anaconda3/envs/capoir/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwa

In [11]:
use_expert_actions = True
discriminator_stack = 5
exp_memory = load_expert_memories(exp_path, load_action=use_expert_actions, n_stack=discriminator_stack)

In [12]:
def one_layer_custom_model(input_shape):
    x_input = Input(shape=input_shape, name='disc_common_input')
    x = Dense(128, activation='relu')(x_input)
    x = Dense(128, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=x_input, outputs=x)
    return model


In [13]:
irl_net_architecture = il_networks.irl_discriminator_net(use_custom_network=True,
                                                         state_custom_network=None,
                                                         common_custom_network=one_layer_custom_model,
                                                         define_custom_output_layer=False)

In [14]:
irl_problem = GAIL(rl_problem, exp_memory, lr_disc=1e-5, batch_size_disc=128, epochs_disc=2, val_split_disc=0.1,
                   n_stack_disc=discriminator_stack, net_architecture=irl_net_architecture,
                   use_expert_actions=use_expert_actions)

/home/serch/TFM/CAPOIRL-TF2/IL_Problem/base/il_problem_super.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.expert_traj = np.array([[np.array([self.preprocess(o) for o in x[0]]), x[1]] for x in expert_traj])


In [15]:
irl_problem.solve(200, render=False, max_step_epi=100, render_after=10)

Episode:  1 Epochs:  128  Reward: 80.9 Smooth Reward: 81.0  Epsilon: 0.9000
Episode:  2 Epochs:  128  Reward: 81.0 Smooth Reward: 81.0  Epsilon: 0.9000
Episode:  3 Epochs:  128  Reward: 80.9 Smooth Reward: 81.0  Epsilon: 0.9000
Episode:  4 Epochs:  128  Reward: 80.9 Smooth Reward: 81.0  Epsilon: 0.9000
Episode:  5 Epochs:  128  Reward: 81.0 Smooth Reward: 81.0  Epsilon: 0.9000
Episode:  6 Epochs:  128  Reward: 81.0 Smooth Reward: 81.0  Epsilon: 0.9000
Episode:  7 Epochs:  128  Reward: 81.0 Smooth Reward: 81.0  Epsilon: 0.9000
Episode:  8 Epochs:  128  Reward: 81.0 Smooth Reward: 81.0  Epsilon: 0.9000
Training discriminator
epoch 1	 loss  1.4201 mean_squared_error 0.2629 val_loss  1.3830 val_mean_squared_error 0.2583
epoch 2	 loss  1.3634 mean_squared_error 0.2545 val_loss  1.3388 val_mean_squared_error 0.2516
Actor loss 0.6343714 0
Critic loss 1.247703 0
Episode:  9 Epochs:  128  Reward: 75.1 Smooth Reward: 78.0  Epsilon: 0.8730
Episode:  10 Epochs:  128  Reward: 75.1 Smooth Reward: 78

Training discriminator
epoch 1	 loss  1.0775 mean_squared_error 0.2028 val_loss  1.0785 val_mean_squared_error 0.2022
epoch 2	 loss  1.0779 mean_squared_error 0.2015 val_loss  1.0748 val_mean_squared_error 0.2010
Actor loss -0.071480766 9
Critic loss 0.016517777 9
Episode:  81 Epochs:  128  Reward: 64.6 Smooth Reward: 64.9  Epsilon: 0.6637
Episode:  82 Epochs:  128  Reward: 64.6 Smooth Reward: 64.9  Epsilon: 0.6637
Episode:  83 Epochs:  128  Reward: 64.9 Smooth Reward: 64.9  Epsilon: 0.6637
Episode:  84 Epochs:  128  Reward: 65.0 Smooth Reward: 64.9  Epsilon: 0.6637
Episode:  85 Epochs:  128  Reward: 64.9 Smooth Reward: 64.9  Epsilon: 0.6637
Episode:  86 Epochs:  128  Reward: 64.8 Smooth Reward: 64.9  Epsilon: 0.6637
Episode:  87 Epochs:  128  Reward: 65.1 Smooth Reward: 64.9  Epsilon: 0.6637
Episode:  88 Epochs:  128  Reward: 64.6 Smooth Reward: 64.9  Epsilon: 0.6637
Training discriminator
epoch 1	 loss  1.0684 mean_squared_error 0.2004 val_loss  1.0660 val_mean_squared_error 0.1998
e

KeyboardInterrupt: 

In [16]:
rl_problem.test(10, True)

/home/serch/anaconda3/envs/capoir/lib/python3.7/site-packages/gym/envs/atari/environment.py:257: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  "We strongly suggest supplying `render_mode` when "


Test episode:  1 Epochs:  496  Reward: 0.0 Smooth Reward: 0.0  Epsilon: 0.6057
Test episode:  2 Epochs:  508  Reward: 0.0 Smooth Reward: 0.0  Epsilon: 0.6057


KeyboardInterrupt: 